In [1]:
import numpy as np
from dynamicSceneGenerator import DynamicSceneGenerator
from visualize import visualize, visualize_camera_gif, visualize_projections, visualize_bounding_boxes
from datatypes.virtualCamera import VirtualCamera
from datatypes.boundingBox import BoundingBox

In [2]:
def calculate_camera_position_WRF(focal_length, width_of_sensor, largest_radius, path_centre, camera_height):
    angle_of_view = 2*np.arctan(width_of_sensor/(2*focal_length)) # In radians
    d = largest_radius/np.tan(angle_of_view/2)
    # Assume camera parallel to x axis and always pointing towards the centre of the path circle
    x = d+path_centre[0]
    y = path_centre[1]
    z = camera_height
    return np.array([x,y,z])

def calculate_camera_pitch(camera_position):
    alpha = np.arctan(camera_position[2]/camera_position[0]) # arctan(height/distance)
    return alpha

In [3]:
def create_and_place_simple_legacy_camera(largest_radius, path_centre): # This should maybe get a better name, and should be moved somewhere?
    '''
    Function for placing a Simple legacy photo camera in the dynamic scene
    '''
    # NB! Make sure everything is in meters
    focal_length = 50*10**-3
    image_bounds = (3600, 2400) # Pixels (x,y)
    film_size = (36*10**-3, 24*10**-3)
    px = film_size[0]/image_bounds[0]
    py = film_size[1]/image_bounds[1]
    principal_point = (image_bounds[0]/2,image_bounds[1]/2)
    width_of_sensor = 36*10**-3 # Width of sensor
    camera_height = 60 # metre

    position_WRF = calculate_camera_position_WRF(focal_length, width_of_sensor, largest_radius, path_centre, camera_height)
    roll = 0
    yaw = np.pi
    pitch = calculate_camera_pitch(position_WRF)
    
    camera = VirtualCamera(position_WRF, roll, yaw, pitch, focal_length, px, py, principal_point, image_bounds)

    return camera

In [4]:
def project_all_points(camera, vessels):
    all_projected_points = {}
    for t in vessels[0].get_track().get_time_stamps():
        points = [vessel.calculate_3D_cornerpoints(t) for vessel in vessels]
        projected_points = [camera.project_points(vessel_points) for vessel_points in points]
        all_projected_points[t] = projected_points
    return all_projected_points

# Where should we save the projected points? A separate class, in the dynamic scene?
# And also the functions in this file should be in a separate class?

In [5]:
def check_bb_overlap(bb1, bb2):
    # How do we want to represent the BBs?
    # Change this according to how the inputted BBs are structured: we need max x, y and min x, y for both BB
    bb1_xmin = bb1.centre[0] - bb1.width/2
    bb1_ymin = bb1.centre[1] - bb1.height/2
    bb1_xmax = bb1.centre[0] + bb1.width/2
    bb1_ymax = bb1.centre[1] + bb1.height/2
    bb2_xmin = bb2.centre[0] - bb2.width/2
    bb2_ymin = bb2.centre[1] - bb2.height/2
    bb2_xmax = bb2.centre[0] + bb2.width/2
    bb2_ymax = bb2.centre[1] + bb2.height/2

    if bb1_xmin >= bb2_xmax or bb1_xmax <= bb2_xmin or bb1_ymin >= bb2_ymax or bb1_ymax <= bb2_ymin:
        return False
    return True

In [6]:
def create_bound_boxes(projected_points):
    bbs = []
    for vessel in projected_points:
        if vessel.size > 0:
            vessel_x = np.array([point.image_coordinate[0] for point in vessel])
            vessel_y = np.array([point.image_coordinate[1] for point in vessel])
            vessel_depth = np.array([point.depth for point in vessel])
            max_x = np.max(vessel_x)
            min_x = np.min(vessel_x)
            max_y = np.max(vessel_y)
            min_y = np.min(vessel_y)
            width = max_x-min_x
            height = max_y - min_y
            centre = [min_x+width/2, min_y+height/2]
            depth = np.average(vessel_depth) #OBS which depth should we use
            bounding_box = BoundingBox(None, centre, width, height, depth)
            # Make sure these are in the correct order as in master thesis
            bbs.append(bounding_box)
    return bbs

def remove_covered_bbs(bounding_boxes):
    bbs = []
    sorted_bbs = sorted(bounding_boxes, key=lambda bb: bb.depth, reverse=True)
    for i in range(len(bounding_boxes)-1):
        overlap = False
        for j in range(i+1, len(bounding_boxes)):
            if check_bb_overlap(sorted_bbs[i], sorted_bbs[j]):
                overlap = True
        if not overlap:
            bbs.append(sorted_bbs[i])
    bbs.append(sorted_bbs[-1])
    return bbs


In [7]:
def remove_covered_bbs(bounding_boxes):
    bbs = []
    sorted_bbs = sorted(bounding_boxes, key=lambda bb: bb.depth, reverse=True)
    for i in range(len(bounding_boxes)-1):
        overlap = False
        for j in range(i+1, len(bounding_boxes)):
            if check_bb_overlap(sorted_bbs[i], sorted_bbs[j]):
                overlap = True
        if not overlap:
            bbs.append(sorted_bbs[i])
    bbs.append(sorted_bbs[-1])
    return bbs

In [8]:
def check_nr_of_points_covered(bb1, bb2):
    if bb2.get_xmin() < bb1.get_xmin() and bb2.get_xmax() > bb1.get_xmax() and bb2.get_ymax() > bb1.get_ymax() and bb2.get_ymin() > bb1.get_ymin() and bb2.get_ymin() < bb1.get_ymax():
        height = bb2.get_ymin() - bb1.get_ymin()
        centre = [bb1.centre[0], bb1.get_ymin()+height/2]
        bb = BoundingBox(bb1.vesselID, centre, bb1.width, height, bb1.depth)
        return bb
    
    elif bb2.get_xmin() < bb1.get_xmin() and bb2.get_xmax() > bb1.get_xmax() and bb2.get_ymax() < bb1.get_ymax() and bb2.get_ymax() < bb1.get_ymin() and bb2.get_ymin() < bb1.get_ymin():
        height = bb1.get_ymax() - bb2.get_ymax()
        centre = [bb1.centre[0], bb2.get_ymax() + height/2]
        bb = BoundingBox(bb1.vesselID, centre, bb1.width, height, bb1.depth)
        return bb
    
    elif bb2.get_ymin() < bb1.get_ymin() and bb2.get_ymax() > bb1.get_ymax() and bb2.get_xmin() < bb1.get_xmin() and bb2.get_xmax() > bb1.get_xmin() and bb2.get_xmax() < bb1.get_xmax():
        width = bb1.get_xmax() - bb2.get_xmax()
        centre = [bb2.get_xmax() + width/2, bb1.centre[1]]
        bb = BoundingBox(bb1.vesselID, centre, width, bb1.height, bb1.depth)
        return bb
    
    elif bb2.get_ymin() < bb1.get_ymin() and bb2.get_ymax() > bb1.get_ymax() and bb2.get_xmax() > bb1.get_xmax() and bb2.get_xmin() < bb1.get_xmax() and bb2.get_xmin() > bb1.get_xmin():
        width = bb2.get_xmin() - bb1.get_xmin()
        centre = [bb1.get_xmin() + width/2, bb1.centre[1]]
        bb = BoundingBox(bb1.vesselID, centre, width, bb1.height, bb1.depth)
        return bb
    
    else:
        return bb1

In [9]:
def check_bb_merge(bb1, bb2):
    bb1_xmin = bb1.centre[0] - bb1.width/2
    bb1_ymin = bb1.centre[1] - bb1.height/2
    bb1_xmax = bb1.centre[0] + bb1.width/2
    bb1_ymax = bb1.centre[1] + bb1.height/2
    bb2_xmin = bb2.centre[0] - bb2.width/2
    bb2_ymin = bb2.centre[1] - bb2.height/2
    bb2_xmax = bb2.centre[0] + bb2.width/2
    bb2_ymax = bb2.centre[1] + bb2.height/2

    if bb1_xmin > bb2_xmax or bb1_xmax < bb2_xmin or bb1_ymin > bb2_ymax or bb1_ymax < bb2_ymin:
        return False
    return True

In [10]:
def create_bbs_for_overlapping_area(bb1, bb2):
    #x 
    x_xmin = np.max(np.array([bb1.get_xmin(), bb2.get_xmin()]))
    x_xmax = np.min(np.array([bb1.get_xmax(), bb2.get_xmax()]))
    x_ymin = np.max(np.array([bb1.get_ymin(), bb2.get_ymin()]))
    x_ymax = np.min(np.array([bb1.get_ymax(), bb2.get_ymax()]))

    width_x = x_xmax-x_xmin
    height_x = x_ymax - x_ymin
    centre_x = [x_xmin+width_x/2, x_ymin+height_x/2]

    #y
    y_xmin = np.min(np.array([bb1.get_xmin(), bb2.get_xmin()]))
    y_xmax = np.max(np.array([bb1.get_xmax(), bb2.get_xmax()]))
    y_ymin = np.min(np.array([bb1.get_ymin(), bb2.get_ymin()]))
    y_ymax = np.max(np.array([bb1.get_ymax(), bb2.get_ymax()]))

    width_y = y_xmax-y_xmin
    height_y = y_ymax - y_ymin
    centre_y = [y_xmin+width_y/2, y_ymin+height_y/2]


    depth = np.min(np.array([bb1.depth, bb2.depth]))

    new_bb_x = BoundingBox(None, centre_x, width_x, height_x, depth)
    new_bb_y = BoundingBox(None, centre_y, width_y, height_y, depth)
    return new_bb_x, new_bb_y

In [11]:
def find_all_bbs_including_overlaps(bbs):
    bbs_cover = []
    if len(bbs)>1:
        for i in range(len(bbs)-1):
            for j in range(i+1, len(bbs)):
                if check_bb_merge(bbs[i], bbs[j]):
                    bb1, bb2 = create_bbs_for_overlapping_area(bbs[i], bbs[j])
                    bbs_cover.append(bb1)
                    bbs_cover.append(bb2)
            bbs_cover.append(bbs[i])
    else:
        bbs_cover.append(bbs[0])
    return bbs_cover

In [12]:
def check_fully_covered(bb1, bb2):
    bb1_xmin = bb1.centre[0] - bb1.width/2
    bb1_ymin = bb1.centre[1] - bb1.height/2
    bb1_xmax = bb1.centre[0] + bb1.width/2
    bb1_ymax = bb1.centre[1] + bb1.height/2
    bb2_xmin = bb2.centre[0] - bb2.width/2
    bb2_ymin = bb2.centre[1] - bb2.height/2
    bb2_xmax = bb2.centre[0] + bb2.width/2
    bb2_ymax = bb2.centre[1] + bb2.height/2

    if bb1_xmin >= bb2_xmin and bb1_xmax <= bb2_xmax and bb1_ymin >= bb2_ymin and bb1_ymax <= bb2_ymax:
        return True
    return False

In [13]:
def find_new_bb(bb1, bbs_covering_bb1):
    bbs_cover = find_all_bbs_including_overlaps(bbs_covering_bb1)
    sorted_bbs = sorted(bbs_cover, key=lambda bb: bb.depth, reverse=True)
    for i in range(len(sorted_bbs)):
        bb1 = check_nr_of_points_covered(bb1, sorted_bbs[i])
    return bb1

In [14]:
def handle_covered_bbs(bounding_boxes):
    bbs = []
    sorted_bbs = sorted(bounding_boxes, key=lambda bb: bb.depth, reverse=True)
    for i in range(len(bounding_boxes)-1):
        bb1 = sorted_bbs[i]
        fully_covered = False
        covering_bbs = []
        for j in range(i+1, len(bounding_boxes)):
            if check_fully_covered(bb1, sorted_bbs[j]):
                fully_covered = True
            elif check_bb_overlap(bb1, sorted_bbs[j]):
                covering_bbs.append(sorted_bbs[j])
        if len(covering_bbs) > 0:
            bb1 = find_new_bb(bb1, covering_bbs)
        if not fully_covered:
            bbs.append(bb1)
    bbs.append(sorted_bbs[-1])
    return bbs

In [15]:
def create_all_bbs(all_projected_points):
    all_bbs = {}
    for t in all_projected_points.keys():
        temp=create_bound_boxes(all_projected_points[t])
        all_bbs[t] = handle_covered_bbs(temp)
    return all_bbs

In [16]:
# Generate dynamic scene with random tracks
dsg = DynamicSceneGenerator()
dsg.set_random_vessels(6)
dsg.generate_random_tracks()
vessels = dsg.get_vessels()
visualize(vessels)

In [17]:
camera = create_and_place_simple_legacy_camera(dsg.get_larges_radius(), dsg.get_path_centre())
visualize_camera_gif(camera, vessels)

In [18]:
all_projected_points = project_all_points(camera, vessels)
visualize_projections(all_projected_points, camera.image_bounds)

In [19]:
all_bbs = create_all_bbs(all_projected_points)
# With points in BBs
visualize_bounding_boxes(all_bbs, camera.image_bounds, projected_points=all_projected_points, show_projected_points=False)
# Without
#visualize_bounding_boxes(all_bbs, camera.image_bounds)
